In [1]:
# Import Libraries
import pandas as pd

In [7]:
my_spotify_data = pd.read_csv("data/my_spotify_data.csv", encoding='latin-1', low_memory=False)

In [8]:
my_spotify_data

,track_name,artist_name,album_name,spotify_track_uri,skipped,date,time,min_played
0,Heaven - Originally Performed By DJ Sammy & Yanou,It's A Cover Up,Dance Anthems,spotify:track:5V2u2roMAFN97vgTt01OlG,TRUE,9/6/2012,19:45:53,1.250816667
1,NaN,NaN,NaN,NaN,TRUE,9/6/2012,19:47:17,1.415833333
2,I Can Do Anything,3OH!3,Streets Of Gold,spotify:track:5jyDUXlAVUh4XJEJIJgapB,TRUE,9/6/2012,19:47:39,0.135666667
3,Sail,AWOLNATION,Megalithic Symphony,spotify:track:7ueP5u2qkdZbIPN2YA6LR0,TRUE,9/6/2012,19:48:20,0.041016667
4,Remember the Name (feat. Styles of Beyond),Fort Minor,Petrified / Remember the Name,spotify:track:3VrI4JYoS75hyLPwG6Z9tn,TRUE,9/6/2012,19:49:04,0.744916667
...,...,...,...,...,...,...,...,...
328947,Sunday Candy,Nico Segal,Surf,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,NaN,6/4/2019,10:15:20,3.460583333
328948,Sunday Candy,Nico Segal,Surf,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,NaN,6/4/2019,10:17:08,0.015466667
328949,CAN'T GET OVER YOU (feat. Clams Casino),Joji,BALLADS 1,spotify:track:39oeKRgsLeynDIbWuXyA47,NaN,6/4/2019,10:18:54,1.7846
328950,Banana Pancakes,Jack Johnson,In Between Dreams,spotify:track:451GvHwY99NKV4zdKPRWmv,NaN,6/4/2019,10:22:07,3.198433333
